The script below plots the exponential and linear fits of each cell's response over Blocks(averaged over first 3 Flashes within a Block). This is plotted separately for each Image.

Further below, the script plots a histogram of the distribution of fit scores.


To understand input csv file, please see add_FlashRepeats_Block_BlockMeanResp.ipynb

Note/Next step: there should be a more efficient way to load the fit_df to extract the fit parameters instead of re-calculating them.

August 2018
Created by M. Sol Bernardez Sarria in collaboration with project teammates and SWDB_2018 TAs

In [ ]:
import matplotlib
%matplotlib notebook

import os
import numpy as np
import pandas as pd

import sys

import matplotlib as mpl
import matplotlib.pyplot as plt

# this makes print statements complaint with python 3 formatting
from __future__ import print_function

# from datetime import datetime

# #Behavior
# from visual_behavior.ophys.dataset.visual_behavior_ophys_dataset import VisualBehaviorOphysDataset
# from visual_behavior.ophys.response_analysis.response_analysis import ResponseAnalysis 

# # Local of the event drive path
# event_drive_path_AWS = '/data/dynamic-brain-workshop/visual_behavior_events' #AWS Location
# drive_path =  '/data/dynamic-brain-workshop/visual_behavior'

#from functions developed by/with our group
import oBehave.helperfunctions as obhf
import oBehave.behavior_anaylsis as obba
import oBehave.perCell_functions as obpc

In [ ]:
filename = ('fit_df:ALL.pkl')
fit_df = pd.read_pickle(filename)
fit_df.head()

In [ ]:
filename = ('data_manifest.pkl')
data_inv = pd.read_pickle(filename)
data_inv.head()

In [ ]:
from scipy.optimize import curve_fit

def fitExp(x,a,b,c):
    return a * np.exp(-b * x) + c

def fitLin(x,a,b):
    return a + b*x 

def r_sq(y, x, f, popt):    
    ss_res = np.sum((y- f(x, *popt))**2)
    ss_tot = np.sum((y-np.mean(y))**2)
    r_squared = 1 - (ss_res / ss_tot)
    return r_squared

In [ ]:
expID ='639438856'
tfilename = ('data_wBlock_mean:', str(expID), '.csv')
filename = ''.join(tfilename)
exp_df = pd.read_csv(filename, index_col= 0)
exp_df.head()

In [ ]:
for cell in exp_df['cell'].unique():
    fig, ax = plt.subplots(nrows=2, ncols=4, sharex=True, sharey=True)
    ax = ax.flatten()
    for n, im in enumerate(exp_df['image_name'].unique()):
        ty = exp_df[(exp_df.cell.values == cell)&(exp_df.image_name.values == im)&(exp_df.repeats.values == 1)]
        y = ty.three_repeats_mr.values
        x0 = np.arange(0,len(y),1)
        check = np.isnan(y)
        if check.any():
            print('cell', cell, im, 'has nan' )
            continue
        else:
            try:
                poptE, pcovE = curve_fit(fitExp, x0, y)
                poptL, pcovL = curve_fit(fitLin, x0, y)
                R_sq_Exp =r_sq(y, x0, fitExp, poptE)
                R_sq_Lin =r_sq(y, x0, fitLin, poptL)
                ax[n].plot(x0, y, 'k.')
                ax[n].plot(x0, fitExp(x0, *poptE), 'b-')
                ax[n].plot(x0, fitLin(x0, *poptL), 'r-')
                ax[n].set_title(im)

            except RuntimeError:
                poptE, pcovE = np.nan, np.nan
                poptL, pcovL = np.nan, np.nan
                R_sq_Exp = np.nan
                R_sq_Lin = np.nan
                    
    tfilename = ('fit:', str(expID),'_cell', str(cell), '.png')
    filename = ''.join(tfilename)
    fig.savefig(filename)
                


In [ ]:
#Plotting fit scores to employ eye-ball test
fit_distLinA = fit_df[fit_df.session_type.values == 'behavior_session_A'].r_sq_lin.values
fit_distExpA = fit_df[fit_df.session_type.values == 'behavior_session_A'].r_sq_exp.values
fit_distLinB = fit_df[fit_df.session_type.values == 'behavior_session_B'].r_sq_lin.values
fit_distExpB = fit_df[fit_df.session_type.values == 'behavior_session_B'].r_sq_exp.values
fit_distLinC = fit_df[fit_df.session_type.values == 'behavior_session_C'].r_sq_lin.values
fit_distExpC = fit_df[fit_df.session_type.values == 'behavior_session_C'].r_sq_exp.values
fit_distLinD = fit_df[fit_df.session_type.values == 'behavior_session_D'].r_sq_lin.values
fit_distExpD = fit_df[fit_df.session_type.values == 'behavior_session_D'].r_sq_exp.values

fit_distLinnotA = fit_df[~fit_df['session_type'].str.contains('A')].r_sq_lin.values
fit_distExpnotA = fit_df[~fit_df['session_type'].str.contains('A')].r_sq_exp.values

In [ ]:
#Comment or uncomment below what you want to plot

colorlist = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.figure()
# plt.hist(fit_distLinA, bins=np.linspace(0,1,50), color ='r', alpha = 0., normed=True,)
plt.hist(fit_distExpA, bins=np.linspace(0,1,50), color = '#1f77b4', alpha = 0.4, normed=True)
# plt.hist(fit_distLinnotA, bins=np.linspace(0,1,50), color ='m', alpha = 0.4, normed=True,)
# plt.hist(fit_distExpnotA, bins=np.linspace(0,1,50), color = '#ff7f0e', alpha = 0.4, normed=True)
plt.hist(fit_distExpB, bins=np.linspace(0,1,50), color = '#ff7f0e', alpha = 0.4, normed=True)
plt.hist(fit_distExpC, bins=np.linspace(0,1,50), color = '#2ca02c', alpha = 0.4, normed=True)
plt.hist(fit_distExpD, bins=np.linspace(0,1,50), color = '#d62728', alpha = 0.4, normed=True)

plt.xlim([0.3, 0.9]) #use when zooming in
plt.ylim([0, 1]) #use when zooming in

plt.xlabel('R_squared')
plt.ylabel('% Population per bin')
         
# plt.legend(['Linear_A', 'Exponential_A', 'Linear_notA', 'Exponential_notA'])
plt.legend(['Exponential_A', 'Exponential_B', 'Exponential_C', 'Exponential_D'])
# plt.legend(['Exponential_A', 'Exponential_notA'])
# tfilename = ('zoomfit_distrib_AnotA.png')
# tfilename = ('fit_distrib_AnotA.png')
# tfilename = ('fit_distrib_ABCD.png')
tfilename = ('zoomfit_distrib_ABCD.png')
filename = ''.join(tfilename)
plt.savefig(filename)



